# Retail Fashion ML/AI Bot - Supreme - Python Tutorial

Though seems like a different world if You are into fashion, skateboarding, motorcycles, music and generally street culture its likely that You know how important is 11:00am on Thursdays, 11:00 on Tuesdays and Sundays drop reveals. Supreme is #1 brand right now and their online drops sell out in matter of seconds. Here are some statistics: US, Europe and Japan.

I've been following and participating in community in various ways and it never ceases to amaze me, especially the people who are into it and amount of energy that flows when coping on Lafayette in Manhattan or wherever You are. And this is my way of giving back. I've used this bot for at least 10 times over the past 2-3 months, and its able to buy in under 2 seonds, so no matter how hyped good is, I am sure You can also get it.

The idea is the following:

0. Specify what items You want based on category, name, color and size (check drop reveals on www.supremecommunity.com)
1. Monitor www.supremenewyork.com for a change in products on 11:00 Thursday based on epoch (usually its on time, but not always, so have another check like number of unique links in 2-3 categories)
2. Get all the links in category You look for (BeutifulSoup ftw!)
3. Match the name, color and size of item You look for (pythonic way!)
4. Get it into cart (Selenium!)
5. Checkout (pay!)
6. Be cool, be supreme ;)

For people not into programming here is a simple windows and linux/mac application that You can download.

In [ ]:
#Load all the libraries 
import requests
import httplib2
from bs4 import BeautifulSoup, SoupStrainer
from PIL import Image, ImageChops
import time
import math, operator
import operator
from selenium import webdriver
import time
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
import warnings
warnings.filterwarnings('ignore')

Replicating human online is easiest done with Selenium library. As You can imagine all the tasks one can think of are available (select list, click button, fill form, etc). Bot starts with selecting webdriver which I chose to be Chrome, though You can use any driver: firefox, phantomJS, etc. 

Please note: checkout process with selenium can be done faster with auto complete extension in browser. But webdriver doesnt load with extensions so I'll leave code for manual import. Also note that it loads blank, so if You want to use it, You should load everything, perform dummy checkout and save the checkout data in extension, and start bot at 11:00 in the same process.

In [ ]:
chrome_options = Options()
#chrome_options.add_extension('7.8.0_0.crx') #Uncomment if you use 
driver = webdriver.Chrome("chromedriver",chrome_options=chrome_options)

Bot is divided into 4 logical parts:

In [ ]:
#1 Get link of Your item

def textwise(link,desTag,col):
    r=requests.get(link)
    soup=BeautifulSoup(r.content)
    if col == 1:
        link = soup.find(itemprop="name")
    else:
        link = soup.find(itemprop="model")
    if link is not None:
        link1 = [s for s in link if desTag in s]
        if link1 is not None:
            if len(link1) > 0:
                return 1
            else:
                return 0
        else:
            return 0
    else:
        return 0

In [ ]:
#2 Get it into cart
def getIt(l1, matchTag,dTag1,dTag2,size):
    matching = [s for s in l1 if matchTag in s]
    matching = [s for s in matching if "all" not in s]
    if len(l1) + len(matching) != 283:
        for aLink in matching:
            txtExtract = textwise(''.join(["http://www.supremenewyork.com",aLink]), dTag1,2)
            if txtExtract == 1 and dTag2 != None:
                txtExtractCol = textwise(''.join(["http://www.supremenewyork.com",aLink]), dTag2,1)
                if txtExtractCol == 1:
                    mostTxt = ''.join(["http://www.supremenewyork.com",aLink])
                    quickOut(mostTxt,size=1)
                    print(mostTxt)
                    break
                else:
                    print("not the color")
            else if txtExtract == 1:
                    mostTxt = ''.join(["http://www.supremenewyork.com",aLink])
                    quickOut(mostTxt,size)
                    print(mostTxt)
                    break
            else:
                print("not the model")
                        #print('OneMore + ',''.join(["http://www.supremenewyork.com",aLink]))
        print('Done!')
    else:
        print('Ciao')

In [ ]:
#3 Checkout
def quickOut(che,sizes):
        driver.get(che)
        if sizes == 1:
            state = driver.find_element_by_id("s").send_keys("Small")
        if sizes == 2:
            state = driver.find_element_by_id("s").send_keys("Medium")
        if sizes == 3:
            state = driver.find_element_by_id("s").send_keys("Large")
        driver.find_element_by_name("commit").click()
        time.sleep(0.09)

In [ ]:
# running operation
t1 = 0
dTag1 = ['Tagless','Court','Arc']
dTag2 = ['Black', 'Black','Blue']
matchTag = ['accessories','sweatshirts','shorts']
size = [3,3,0]
chasing = 0
while t1 == 0:
    if time.time() > 1525359600:
        t1 = 1
        http = httplib2.Http()
        status, response = http.request('http://www.supremenewyork.com/shop/all')
        l1 = []
        for link in BeautifulSoup(response, 'html.parser', parseOnlyThese=SoupStrainer('a')):
            if link.has_attr('href'):
                l1.append(link['href'])
        for k in range(0,len(dTag1)):
            getIt(l1,dTag1[k],dTag2[k],matchTag[k],size[k])
            #getIt(l1,matchTag[k],dTag1[k],dTag2[k])
    else:
        next
driver.get("https://www.supremenewyork.com/checkout")
time.sleep(0.1)
 
#fill form
driver.find_element_by_id("order_billing_name").send_keys("Mister Supreme")
email = driver.find_element_by_id("order_email").send_keys("dev@vladovukovic.com")
tel = driver.find_element_by_id("order_tel").send_keys("+13473201982")
address = driver.find_element_by_id("bo").send_keys("157 25th street")
zip = driver.find_element_by_id("order_billing_zip").send_keys("11232")
city = driver.find_element_by_id("order_billing_city").send_keys("Brooklyn")
# apt = driver.find_element_by_id("oba3").send_keys("test")
#select box
state = driver.find_element_by_id("order_billing_state").send_keys("CA")
usa = driver.find_element_by_id("order_billing_country").send_keys("USA")
#card
cardNumber = driver.find_element_by_id("nnaerb").send_keys("1111222233334444")
#select box
cardDay = driver.find_element_by_id("credit_card_month").send_keys("01")
cardYear = driver.find_element_by_id("credit_card_year").send_keys("2019")
cvv = driver.find_element_by_id("orcer").send_keys("686")
 
#terms
orderTerms = driver.find_element_by_id("order_terms").send_keys(Keys.SPACE)

In order to avoid triggering capture the "last mile" is on Your own, to click checkout and mark the checkbox. If its programmed it will trigger quite long process of proving You are not bot, though Your order is reserver in database so doesnt matter how long it takes You to finish it, it will still work.